In [20]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo

# Mac Users

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

In [3]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

# Windows Users

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# URL of page to be scraped
url = 'https://store.steampowered.com/stats'
browser.visit(url)

In [6]:
#Scrape list of top games from the Steam Stats page
browser.find_link_by_text('View all of the top 100 most-played games').first.click()

C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:501: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


In [7]:
# Append scraped table data into a list
print(browser.is_element_present_by_xpath('//*[@id="detailStats"]', wait_time = 5))

price_table = browser.find_by_xpath('//*[@id="detailStats"]/table/tbody/tr')

current = []
for i in price_table:
    print(i.text)
    current.append(i.text)

True
CURRENT PLAYERS PEAK TODAY   GAME
 
642,276 1,040,051   Counter-Strike: Global Offensive
361,413 678,801   Dota 2
189,677 250,338   Among Us
183,737 196,805   Destiny 2
100,945 117,465   Rocket League
85,806 106,331   Source SDK Base 2013 Multiplayer
79,800 86,333   Team Fortress 2
72,778 435,473   PLAYERUNKNOWN'S BATTLEGROUNDS
71,067 85,441   Rust
67,367 137,120   Grand Theft Auto V
67,116 117,181   Apex Legends
62,077 83,258   ARK: Survival Evolved
53,907 95,794   Tom Clancy's Rainbow Six Siege
49,888 55,958   Phasmophobia
46,803 53,404   Sid Meier's Civilization VI
44,097 44,930   PAYDAY 2
40,101 65,814   Warframe
38,447 50,623   Football Manager 2021
37,270 48,791   Dead by Daylight
35,755 42,976   Terraria
34,558 46,081   Football Manager 2020
32,074 48,519   War Thunder
31,603 38,896   Hearts of Iron IV
29,736 34,088   Total War: WARHAMMER II
29,231 39,874   Garry's Mod
28,280 30,599   Brawlhalla
27,952 30,971   7 Days to Die
27,095 66,512   Wallpaper Engine
26,082 31,563   

In [8]:
browser.quit()

In [9]:
#Display list
print(current)

['CURRENT PLAYERS PEAK TODAY   GAME', ' ', '642,276 1,040,051   Counter-Strike: Global Offensive', '361,413 678,801   Dota 2', '189,677 250,338   Among Us', '183,737 196,805   Destiny 2', '100,945 117,465   Rocket League', '85,806 106,331   Source SDK Base 2013 Multiplayer', '79,800 86,333   Team Fortress 2', "72,778 435,473   PLAYERUNKNOWN'S BATTLEGROUNDS", '71,067 85,441   Rust', '67,367 137,120   Grand Theft Auto V', '67,116 117,181   Apex Legends', '62,077 83,258   ARK: Survival Evolved', "53,907 95,794   Tom Clancy's Rainbow Six Siege", '49,888 55,958   Phasmophobia', "46,803 53,404   Sid Meier's Civilization VI", '44,097 44,930   PAYDAY 2', '40,101 65,814   Warframe', '38,447 50,623   Football Manager 2021', '37,270 48,791   Dead by Daylight', '35,755 42,976   Terraria', '34,558 46,081   Football Manager 2020', '32,074 48,519   War Thunder', '31,603 38,896   Hearts of Iron IV', '29,736 34,088   Total War: WARHAMMER II', "29,231 39,874   Garry's Mod", '28,280 30,599   Brawlhalla',

In [10]:
# Convert list to dataframe
df = pd.DataFrame(current)
df.head(10)

,0
0,CURRENT PLAYERS PEAK TODAY GAME
1,
2,"642,276 1,040,051 Counter-Strike: Global Off..."
3,"361,413 678,801 Dota 2"
4,"189,677 250,338 Among Us"
5,"183,737 196,805 Destiny 2"
6,"100,945 117,465 Rocket League"
7,"85,806 106,331 Source SDK Base 2013 Multiplayer"
8,"79,800 86,333 Team Fortress 2"
9,"72,778 435,473 PLAYERUNKNOWN'S BATTLEGROUNDS"


In [11]:
# Split by the double-space to extract game name from player into a separate column
df1 = df[0].str.split('  ', expand=True)
df1.head(10)

,0,1
0,CURRENT PLAYERS PEAK TODAY,GAME
1,,None
2,"642,276 1,040,051",Counter-Strike: Global Offensive
3,"361,413 678,801",Dota 2
4,"189,677 250,338",Among Us
5,"183,737 196,805",Destiny 2
6,"100,945 117,465",Rocket League
7,"85,806 106,331",Source SDK Base 2013 Multiplayer
8,"79,800 86,333",Team Fortress 2
9,"72,778 435,473",PLAYERUNKNOWN'S BATTLEGROUNDS


In [12]:
# Split player data into separate columns
df2 = df1[0].str.split(' ', expand=True)
df3 = df2.drop([2, 3], axis=1)
df3.columns = [
  'current_player',
  'peak_today'
]
df3.head(10)

,current_player,peak_today
0,CURRENT,PLAYERS
1,,
2,"642,276","1,040,051"
3,"361,413","678,801"
4,"189,677","250,338"
5,"183,737","196,805"
6,"100,945","117,465"
7,"85,806","106,331"
8,"79,800","86,333"
9,"72,778","435,473"


In [13]:
# Display games in a separate dataframe to concatenate with player data
df5=df1.drop([0], axis=1)
df5.columns = [
  'game',
]
df5.head(10)

,game
0,GAME
1,None
2,Counter-Strike: Global Offensive
3,Dota 2
4,Among Us
5,Destiny 2
6,Rocket League
7,Source SDK Base 2013 Multiplayer
8,Team Fortress 2
9,PLAYERUNKNOWN'S BATTLEGROUNDS


In [14]:
# Concatenate player data to game name in a new dataframe
df5 = pd.concat([df3, df5], axis=1)
df6 = df5.iloc[2:]
df6.head(10) 

,current_player,peak_today,game
2,"642,276","1,040,051",Counter-Strike: Global Offensive
3,"361,413","678,801",Dota 2
4,"189,677","250,338",Among Us
5,"183,737","196,805",Destiny 2
6,"100,945","117,465",Rocket League
7,"85,806","106,331",Source SDK Base 2013 Multiplayer
8,"79,800","86,333",Team Fortress 2
9,"72,778","435,473",PLAYERUNKNOWN'S BATTLEGROUNDS
10,"71,067","85,441",Rust
11,"67,367","137,120",Grand Theft Auto V


In [15]:
# Remove leading space from game in order to join with other data source
# df6.set_index('game')
df6['game'] = df6['game'].str.lstrip()
df6

C:\Users\cch31\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,current_player,peak_today,game
2,"642,276","1,040,051",Counter-Strike: Global Offensive
3,"361,413","678,801",Dota 2
4,"189,677","250,338",Among Us
5,"183,737","196,805",Destiny 2
6,"100,945","117,465",Rocket League
...,...,...,...
97,"6,801","7,180",NGU IDLE
98,"6,562","6,715",Risk of Rain 2
99,"6,373","7,894",The Binding of Isaac: Rebirth
100,"6,331","8,745",Football Manager 2019


In [16]:
# Write out file to cvs
df6.to_csv('top_100_steam.csv')

In [32]:
# Read the Kaggle cvs datafile into pandas and filter columns 
steam_catalog = pd.read_csv('steam_games.csv').rename(columns={"name": "game"})
game_info_df = steam_catalog[['game', 'all_reviews', 'release_date', 'developer','popular_tags', 'game_details', 'achievements', 'genre', 'original_price', 'discount_price']]
game_info_df

,game,all_reviews,release_date,developer,popular_tags,game_details,achievements,genre,original_price,discount_price
0,DOOM,"Very Positive,(42,550),- 92% of the 42,550 use...",12-May-16,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...",54.0,Action,$19.99,$14.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(836,608),- 49% of the 836,608 user revi...",21-Dec-17,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats",37.0,"Action,Adventure,Massively Multiplayer",$29.99,NaN
2,BATTLETECH,"Mostly Positive,(7,030),- 71% of the 7,030 use...",24-Apr-18,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...",128.0,"Action,Adventure,Strategy",$39.99,NaN
3,DayZ,"Mixed,(167,115),- 61% of the 167,115 user revi...",13-Dec-18,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...",NaN,"Action,Adventure,Massively Multiplayer",$44.99,NaN
4,EVE Online,"Mostly Positive,(11,481),- 74% of the 11,481 u...",6-May-03,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",Free,NaN
...,...,...,...,...,...,...,...,...,...,...
40828,Rocksmith 2014 Edition – Remastered – Sabaton ...,NaN,12-Feb-19,Ubisoft - San Francisco,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...",NaN,"Casual,Simulation",$2.99,NaN
40829,Rocksmith 2014 Edition – Remastered – Stone Te...,NaN,5-Feb-19,Ubisoft - San Francisco,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...",NaN,"Casual,Simulation",$2.99,NaN
40830,Fantasy Grounds - Quests of Doom 4: A Midnight...,NaN,31-Jul-18,"SmiteWorks USA, LLC","RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...",NaN,"Indie,RPG,Strategy",$7.99,NaN
40831,Mega Man X5 Sound Collection,NaN,24-Jul-18,"CAPCOM CO., LTD",Action,"Single-player,Downloadable Content,Steam Achie...",NaN,Action,$9.99,NaN


In [36]:
# join/merge the two data sources
merge_df = pd.merge(df6, game_info_df, on="game", how="left").fillna('')
merge_df

,current_player,peak_today,game,all_reviews,release_date,developer,popular_tags,game_details,achievements,genre,original_price,discount_price
0,"642,276","1,040,051",Counter-Strike: Global Offensive,"Very Positive,(3,094,223),- 86% of the 3,094,2...",21-Aug-12,"Valve,Hidden Path Entertainment","FPS,Shooter,Multiplayer,Competitive,Action,Tea...","Multi-player,Steam Achievements,Full controlle...",167,"Action,Free to Play",Free to Play,$65.70
1,"361,413","678,801",Dota 2,"Very Positive,(1,015,621),- 85% of the 1,015,6...",9-Jul-13,Valve,"Free to Play,MOBA,Multiplayer,Strategy,e-sport...","Multi-player,Co-op,Steam Trading Cards,Steam W...",,"Action,Free to Play,Strategy",Free to Play,$65.70
2,"189,677","250,338",Among Us,"Very Positive,(106),- 84% of the 106 user revi...",16-Nov-18,Innersloth,"Casual,Multiplayer,Local Multiplayer,Space,Onl...","Multi-player,Online Multi-Player,Local Multi-P...",,Casual,$4.99,
3,"183,737","196,805",Destiny 2,,17-Sep-19,Bungie,"FPS,Multiplayer,Sci-fi,Great Soundtrack,MMORPG...","Single-player,Online Multi-Player,Online Co-op...",,"Action,Adventure",$34.99,
4,"100,945","117,465",Rocket League,"Very Positive,(188,275),*,- 91% of the 188,275...",7-Jul-15,"Psyonix, Inc.","Multiplayer,Soccer,Competitive,Sports,Racing,T...","Single-player,Multi-player,Co-op,Shared/Split ...",88,"Action,Indie,Racing,Sports",$19.99,
...,...,...,...,...,...,...,...,...,...,...,...,...
95,"6,801","7,180",NGU IDLE,,,,,,,,,
96,"6,562","6,715",Risk of Rain 2,"Very Positive,(16,312),- 94% of the 16,312 use...",28-Mar-19,Hopoo Games,"Early Access,Rogue-like,Co-op,Multiplayer,Earl...","Single-player,Online Multi-Player,Online Co-op...",,"Action,Indie,Early Access",$19.99,
97,"6,373","7,894",The Binding of Isaac: Rebirth,"Overwhelmingly Positive,(63,149),- 97% of the ...",4-Nov-14,"Nicalis, Inc.","Rogue-like,Indie,Replay Value,Difficult,Pixel ...","Single-player,Shared/Split Screen,Steam Achiev...",403,Action,$14.99,$35.97
98,"6,331","8,745",Football Manager 2019,"Mostly Positive,(2,566),- 71% of the 2,566 use...",2-Nov-18,Sports Interactive,"Simulation,Sports,Management,Soccer,Football,S...","Single-player,Multi-player,Steam Achievements,...",77,"Simulation,Sports",$49.99,


In [37]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [22]:
# Define database and collection
db = client.steam_top_db
collection = db.games

In [38]:
# Dictionary to be inserted as a MongoDB document
data_dict = merge_df.to_dict("records")

# Insert collection
collection.insert_many(data_dict)


In [39]:
# Display MongoDB records within the collection
games = db.games.find()

for game in games:
    print(game)

{'_id': ObjectId('5fb9a557b4f5725240c8182a'), 'current_player': '642,276', 'peak_today': '1,040,051', 'game': 'Counter-Strike: Global Offensive', 'all_reviews': 'Very Positive,(3,094,223),- 86% of the 3,094,223 user reviews for this game are positive.', 'release_date': '21-Aug-12', 'developer': 'Valve,Hidden Path Entertainment', 'popular_tags': 'FPS,Shooter,Multiplayer,Competitive,Action,Team-Based,e-sports,Tactical,First-Person,PvP,Online Co-Op,Co-op,Strategy,Military,War,Difficult,Trading,Fast-Paced,Realistic,Moddable', 'game_details': 'Multi-player,Steam Achievements,Full controller support,Steam Trading Cards,Steam Workshop,In-App Purchases,Valve Anti-Cheat enabled,Stats', 'achievements': 167.0, 'genre': 'Action,Free to Play', 'original_price': 'Free to Play', 'discount_price': '$65.70 '}
{'_id': ObjectId('5fb9a557b4f5725240c8182b'), 'current_player': '361,413', 'peak_today': '678,801', 'game': 'Dota 2', 'all_reviews': 'Very Positive,(1,015,621),- 85% of the 1,015,621 user reviews 